In [91]:
import pandas as pd, numpy as np

---

# Merge, Join, and Concatenate

pandas provides various facilities for easily combining together Series or DataFrame with various kinds of set logic for the indexes and relational algebra functionality in the case of join / merge-type operations.

---

# Concatenating objects: `pd.concat()`

The `concat()` function (in the main pandas namespace) does all of the heavy lifting of performing concatenation operations along an axis while performing optional set logic (union or intersection) of the indexes (if any) on the other axes. Note that I say “if any” because there is only a single possible axis of concatenation for Series.

Before diving into all of the details of concat and what it can do, here is a simple example:

In [92]:
def mklbl(prefix,n1,n2):
    return ['%s%s' % (prefix, i) for i in range(n1, n2)]

In [93]:
df1 = pd.DataFrame([mklbl('A',0,4),
                    mklbl('B',0,4),
                    mklbl('C',0,4),
                    mklbl('D',0,4)],
                   index=['A','B','C','D'],
                   columns=[0,1,2,3])

In [94]:
df2 = pd.DataFrame([mklbl('A',4,8),
                    mklbl('B',4,8),
                    mklbl('C',4,8),
                    mklbl('D',4,8)],
                   index=['A','B','C','D'],
                   columns=[4,5,6,7])

In [95]:
df3 = pd.DataFrame([mklbl('A',8,12),
                    mklbl('B',8,12),
                    mklbl('C',8,12),
                    mklbl('D',8,12)],
                   index=['A','B','C','D'],
                   columns=[8,9,10,11])

In [96]:
df1 = df1.T
df2 = df2.T
df3 = df3.T

In [97]:
frames = [df1,df2,df3]

In [98]:
result = pd.concat(frames)

In [99]:
result

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3
4,A4,B4,C4,D4
5,A5,B5,C5,D5
6,A6,B6,C6,D6
7,A7,B7,C7,D7
8,A8,B8,C8,D8
9,A9,B9,C9,D9


Like its sibling function on ndarrays, `numpy.concatenate`, `pandas.concat` takes a list or dict of homogeneously-typed objects and concatenates them with some configurable handling of “what to do with the other axes”:

see [pandas doc](https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html)

>pd.concat(objs, axis=0, join='outer', ignore_index=**False**, keys=**None**,
          levels=**None**, names=**None**, verify_integrity=**False**, copy=**True**)

Without a little bit of context many of these arguments don’t make much sense. Let’s revisit the above example. Suppose we wanted to associate specific keys with each of the pieces of the chopped up DataFrame. We can do this using the `keys` argument:

In [100]:
result = pd.concat(frames, keys=['x', 'y', 'z'])

In [101]:
result

A    B    C    D
x 0    A0   B0   C0   D0
  1    A1   B1   C1   D1
  2    A2   B2   C2   D2
  3    A3   B3   C3   D3
y 4    A4   B4   C4   D4
  5    A5   B5   C5   D5
  6    A6   B6   C6   D6
  7    A7   B7   C7   D7
z 8    A8   B8   C8   D8
  9    A9   B9   C9   D9
  10  A10  B10  C10  D10
  11  A11  B11  C11  D11

As you can see (if you’ve read the rest of the documentation), the resulting object’s index has a hierarchical index. This means that we can now select out each chunk by key:

In [102]:
result.loc['y']

,A,B,C,D
4,A4,B4,C4,D4
5,A5,B5,C5,D5
6,A6,B6,C6,D6
7,A7,B7,C7,D7


It’s not a stretch to see how this can be very useful. More detail on this functionality below.

>**Note** It is worth noting that `concat()` (and therefore `append()`) makes a full copy of the data, and that constantly reusing this function can create a significant performance hit. If you need to use the operation over several datasets, use a list comprehension.

### Set logic on the other axes: `join='outer'`, `axis=0`

When gluing together multiple DataFrames, you have a choice of how to handle the other axes (other than the one being concatenated). This can be done in the following two ways:

- Take the union of them all, `join='outer'`. This is the default option as it results in zero information loss.
- Take the intersection, `join='inner'`.

Here is an example of each of these methods. First, the default `join='outer'` behavior:

In [103]:
df4 = pd.DataFrame(  {'B': ['B2', 'B3', 'B6', 'B7'],
                      'D': ['D2', 'D3', 'D6', 'D7'],
                      'F': ['F2', 'F3', 'F6', 'F7']},
                   index=[2,3,6,7])

In [104]:
result = pd.concat([df1, df4], axis=1, sort=False)

In [105]:
result

,A,B,C,D,B,D,F
0,A0,B0,C0,D0,NaN,NaN,NaN
1,A1,B1,C1,D1,NaN,NaN,NaN
2,A2,B2,C2,D2,B2,D2,F2
3,A3,B3,C3,D3,B3,D3,F3
6,NaN,NaN,NaN,NaN,B6,D6,F6
7,NaN,NaN,NaN,NaN,B7,D7,F7


Here is the same thing with `join='inner'`

In [106]:
result = pd.concat([df1, df4], join='inner', axis=1)

In [107]:
result

,A,B,C,D,B,D,F
2,A2,B2,C2,D2,B2,D2,F2
3,A3,B3,C3,D3,B3,D3,F3


Lastly, suppose we just wanted to reuse the exact index *from the original DataFrame*:

In [108]:
result= pd.concat([df1, df4], axis=1).reindex(df1.index)

In [109]:
result

,A,B,C,D,B,D,F
0,A0,B0,C0,D0,NaN,NaN,NaN
1,A1,B1,C1,D1,NaN,NaN,NaN
2,A2,B2,C2,D2,B2,D2,F2
3,A3,B3,C3,D3,B3,D3,F3


Similarly, we could index before the concatenation:

In [110]:
pd.concat([df1, df4.reindex(df1.index)], axis=1)

,A,B,C,D,B,D,F
0,A0,B0,C0,D0,NaN,NaN,NaN
1,A1,B1,C1,D1,NaN,NaN,NaN
2,A2,B2,C2,D2,B2,D2,F2
3,A3,B3,C3,D3,B3,D3,F3


### Concatenating using `append()`

A useful shortcut to `concat()` are the `append()` instance methods on Series and DataFrame. These methods actually predated `concat`. They concatenate along `axis=0`, namely the index:

In [111]:
result = df1.append(df2)

In [112]:
result

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3
4,A4,B4,C4,D4
5,A5,B5,C5,D5
6,A6,B6,C6,D6
7,A7,B7,C7,D7


In the case of DataFrame, the indexes must be disjoint but the columns do not need to be:

In [113]:
df1.append(df4, sort=False)

,A,B,C,D,F
0,A0,B0,C0,D0,NaN
1,A1,B1,C1,D1,NaN
2,A2,B2,C2,D2,NaN
3,A3,B3,C3,D3,NaN
2,NaN,B2,NaN,D2,F2
3,NaN,B3,NaN,D3,F3
6,NaN,B6,NaN,D6,F6
7,NaN,B7,NaN,D7,F7


`append` may take multiple objects to concatenate:

In [114]:
result = df1.append([df2, df3])

In [115]:
result

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3
4,A4,B4,C4,D4
5,A5,B5,C5,D5
6,A6,B6,C6,D6
7,A7,B7,C7,D7
8,A8,B8,C8,D8
9,A9,B9,C9,D9


>**Note** Unlike the `append()` method, which appends to the original list and returns `None`, `append()` here does not modify `df1` and returns its copy with `df2` appended.

### Igonring indexes on the concatenation axis : `ignore_index=True`

For DataFrame objects which don’t have a meaningful index, you may wish to append them and ignore the fact that they may have overlapping, duplicate indexes. To do this, use the `ignore_index` argument:

In [116]:
result = pd.concat([df1, df4], ignore_index=True, sort=False)

In [117]:
result

,A,B,C,D,F
0,A0,B0,C0,D0,NaN
1,A1,B1,C1,D1,NaN
2,A2,B2,C2,D2,NaN
3,A3,B3,C3,D3,NaN
4,NaN,B2,NaN,D2,F2
5,NaN,B3,NaN,D3,F3
6,NaN,B6,NaN,D6,F6
7,NaN,B7,NaN,D7,F7


This is also a valid argument to `DataFrame.append()`:

In [118]:
result = df1.append(df4, ignore_index=True, sort=False)

In [119]:
result

,A,B,C,D,F
0,A0,B0,C0,D0,NaN
1,A1,B1,C1,D1,NaN
2,A2,B2,C2,D2,NaN
3,A3,B3,C3,D3,NaN
4,NaN,B2,NaN,D2,F2
5,NaN,B3,NaN,D3,F3
6,NaN,B6,NaN,D6,F6
7,NaN,B7,NaN,D7,F7


### Concatenating with mixed ndims

You can concatenate a mix of `Series` and `DataFrame` objects. The Series will be transformed to DataFrame with the column name as the name of the Series.

In [120]:
s1 = pd.Series(mklbl('X',0,3), name='X')

In [121]:
result = pd.concat([df1, s1], axis=1).reindex(s1.index)

In [122]:
result

,A,B,C,D,X
0,A0,B0,C0,D0,X0
1,A1,B1,C1,D1,X1
2,A2,B2,C2,D2,X2


>**Note** Since we’re concatenating a `Series` to a DataFrame, we could have achieved the same result with `DataFrame.assign()`. To concatenate an arbitrary number of pandas objects (`DataFrame` or `Series`), use concat.

If unnamed Series are passed they will be numbered consecutively.

In [123]:
s2 = pd.Series(['_0', '_1', '_2', '_3'])

In [124]:
result = pd.concat([df1, s2, s2, s2], axis=1)

In [125]:
result

,A,B,C,D,0,1,2
0,A0,B0,C0,D0,_0,_0,_0
1,A1,B1,C1,D1,_1,_1,_1
2,A2,B2,C2,D2,_2,_2,_2
3,A3,B3,C3,D3,_3,_3,_3


Passing ignore_index=True will drop all name references.

In [126]:
result = pd.concat([df1, s1], axis=1, ignore_index=True)

In [127]:
result

,0,1,2,3,4
0,A0,B0,C0,D0,X0
1,A1,B1,C1,D1,X1
2,A2,B2,C2,D2,X2
3,A3,B3,C3,D3,NaN


### More Concatenating with group keys: `keys=[]`

A fairly common use of the `keys` argument is to override the column names when creating a new `DataFrame` based on existing `Series`. Notice how the default behaviour consists on letting the resulting `DataFrame` inherit the parent `Series`’ name, when these existed.

In [128]:
s3 = pd.Series([0,1,2,3], name='foo')
s4 = pd.Series([0,1,2,3])
s5 = pd.Series([0,1,4,5])

In [129]:
pd.concat([s3, s4, s5], axis=1)

,foo,0,1
0,0,0,0
1,1,1,1
2,2,2,4
3,3,3,5


Through the `keys` argument we can override the existing column names.

In [130]:
pd.concat([s3, s4, s5], axis=1, keys=['red', 'blue', 'yellow'])

,red,blue,yellow
0,0,0,0
1,1,1,1
2,2,2,4
3,3,3,5


In [131]:
pd.concat([s3, s4, s5], axis=1, keys=['red', 'blue', 'yellow'])

,red,blue,yellow
0,0,0,0
1,1,1,1
2,2,2,4
3,3,3,5


Let’s consider a variation of the very first example presented:

In [132]:
result = pd.concat(frames, keys=['x', 'y', 'z'])

In [133]:
result

A    B    C    D
x 0    A0   B0   C0   D0
  1    A1   B1   C1   D1
  2    A2   B2   C2   D2
  3    A3   B3   C3   D3
y 4    A4   B4   C4   D4
  5    A5   B5   C5   D5
  6    A6   B6   C6   D6
  7    A7   B7   C7   D7
z 8    A8   B8   C8   D8
  9    A9   B9   C9   D9
  10  A10  B10  C10  D10
  11  A11  B11  C11  D11

You can also pass a dict to `concat` in which case the dict keys will be used for the keys argument (unless other keys are specified):

In [134]:
pieces = {'x': df1, 'y': df2, 'z': df3}

In [135]:
result = pd.concat( {'x': df1, 'y': df2, 'z': df3}, axis=0 )

In [136]:
result

A    B    C    D
x 0    A0   B0   C0   D0
  1    A1   B1   C1   D1
  2    A2   B2   C2   D2
  3    A3   B3   C3   D3
y 4    A4   B4   C4   D4
  5    A5   B5   C5   D5
  6    A6   B6   C6   D6
  7    A7   B7   C7   D7
z 8    A8   B8   C8   D8
  9    A9   B9   C9   D9
  10  A10  B10  C10  D10
  11  A11  B11  C11  D11

In [137]:
result = pd.concat(pieces, keys=['z', 'y'])

In [138]:
result

A    B    C    D
z 8    A8   B8   C8   D8
  9    A9   B9   C9   D9
  10  A10  B10  C10  D10
  11  A11  B11  C11  D11
y 4    A4   B4   C4   D4
  5    A5   B5   C5   D5
  6    A6   B6   C6   D6
  7    A7   B7   C7   D7

The MultiIndex created has levels that are constructed from the passed keys and the index of the DataFrame pieces:

In [139]:
result.index.levels

FrozenList([['z', 'y'], [4, 5, 6, 7, 8, 9, 10, 11]])

If you wish to specify other levels (as will occasionally be the case), you can do so using the levels argument:

In [140]:
result = pd.concat(pieces, keys=['x','y','z'],
                  levels=[['z','y','x','w']],
                  names=['group_key'])

In [141]:
result

A    B    C    D
group_key                       
x         0    A0   B0   C0   D0
          1    A1   B1   C1   D1
          2    A2   B2   C2   D2
          3    A3   B3   C3   D3
y         4    A4   B4   C4   D4
          5    A5   B5   C5   D5
          6    A6   B6   C6   D6
          7    A7   B7   C7   D7
z         8    A8   B8   C8   D8
          9    A9   B9   C9   D9
          10  A10  B10  C10  D10
          11  A11  B11  C11  D11

In [142]:
result.index.levels

FrozenList([['z', 'y', 'x', 'w'], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]])

This is fairly esoteric, but it is actually necessary for implementing things like `GroupBy` where the order of a categorical variable is meaningful.

### Appending rows to a DataFrame

While not especially efficient (since a new object must be created), you can append a single row to a DataFrame by passing a Series or dict to append, which returns a new DataFrame as above.

In [143]:
s2 = pd.Series(['X0', 'X1', 'X2', 'X3'], index=['A', 'B', 'C', 'D'])

In [144]:
result = df1.append(s2, ignore_index=True)

In [145]:
result

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3
4,X0,X1,X2,X3


You **should** use `ignore_index=True` with this method to instruct DataFrame to discard its index. If you wish to preserve the index, you should construct an appropriately-indexed DataFrame and append or concatenate those objects.

You can also pass a list of dicts or Series:

In [147]:
dicts = [{'A':1, 'B':2, 'C':3, 'X':4},
         {'A':5, 'B':6, 'C':7, 'Y':8}]

In [149]:
result = df1.append(dicts, ignore_index=True, sort=True)

In [150]:
result

,A,B,C,D,X,Y
0,A0,B0,C0,D0,NaN,NaN
1,A1,B1,C1,D1,NaN,NaN
2,A2,B2,C2,D2,NaN,NaN
3,A3,B3,C3,D3,NaN,NaN
4,1,2,3,NaN,4.0,NaN
5,5,6,7,NaN,NaN,8.0


---
# Database-style DataFrame or named Series joining/merging

pandas has full-featured, high performance in-memory join operations idiomatically very similar to relational databases like SQL. These methods perform significantly better (in some cases well over an order of magnitude better) than other open source implementations (like `base::merge.data.frame` in R). The reason for this is careful algorithmic design and the internal layout of the data in DataFrame.

See the [cookbook](https://pandas.pydata.org/pandas-docs/stable/user_guide/cookbook.html#cookbook-merge) for some advanced strategies.

Users who are familiar with SQL but new to pandas might be interested in a comparison with SQL.

pandas provides a single function, `merge()`, as the entry point for all standard database join operations between DataFrame or named Series objects:

```
pd.merge(left, right, how='inner', on=None, left_on=None, right_on=None,
         left_index=False, right_index=False, sort=True,
         suffixes=('_x', '_y'), copy=True, indicator=False,
         validate=None)
```

- left: A DataFrame or named Series object.

- right: Another DataFrame or named Series object.

- on: Column or index level names to join on. Must be found in both the left and right DataFrame and/or Series objects. If not passed and left_index and right_index are False, the intersection of the columns in the DataFrames and/or Series will be inferred to be the join keys.

- left_on: Columns or index levels from the left DataFrame or Series to use as keys. Can either be column names, index level names, or arrays with length equal to the length of the DataFrame or Series.

- right_on: Columns or index levels from the right DataFrame or Series to use as keys. Can either be column names, index level names, or arrays with length equal to the length of the DataFrame or Series.

- left_index: If True, use the index (row labels) from the left DataFrame or Series as its join key(s). In the case of a DataFrame or Series with a MultiIndex (hierarchical), the number of levels must match the number of join keys from the right DataFrame or Series.

- right_index: Same usage as left_index for the right DataFrame or Series

- how: One of 'left', 'right', 'outer', 'inner'. Defaults to inner. See below for more detailed description of each method.

- sort: Sort the result DataFrame by the join keys in lexicographical order. Defaults to True, setting to False will improve performance substantially in many cases.

- suffixes: A tuple of string suffixes to apply to overlapping columns. Defaults to ('_x', '_y').

- copy: Always copy data (default True) from the passed DataFrame or named Series objects, even when reindexing is not necessary. Cannot be avoided in many cases but may improve performance / memory usage. The cases where copying can be avoided are somewhat pathological but this option is provided nonetheless.

- indicator: Add a column to the output DataFrame called _merge with information on the source of each row. _merge is Categorical-type and takes on a value of left_only for observations whose merge key only appears in 'left' DataFrame or Series, right_only for observations whose merge key only appears in 'right' DataFrame or Series, and both if the observation’s merge key is found in both.

- validate : string, default None. If specified, checks if merge is of specified type.

  - “one_to_one” or “1:1”: checks if merge keys are unique in both left and right datasets.
  - “one_to_many” or “1:m”: checks if merge keys are unique in left dataset.
  - “many_to_one” or “m:1”: checks if merge keys are unique in right dataset.
  - “many_to_many” or “m:m”: allowed, but does not result in checks.

>**Note** Support for specifying index levels as the `on`, `left_on`, and `right_on` parameters was added in version 0.23.0. Support for merging named `Series` objects was added in version 0.24.0.

The return type will be the same as `left`. If `left` is a DataFrame or named Series and `right` is a subclass of DataFrame, the return type will still be DataFrame.

merge is a function in the pandas namespace, and it is also available as a DataFrame instance method merge(), with the calling DataFrame being implicitly considered the `left` object in the join.

The related `join()` method, uses `merge` internally for the index-on-index (by default) and column(s)-on-index join. If you are joining on index only, you may wish to use `DataFrame.join` to save yourself some typing.

## Brief primer on merge methods (relational algebra)

Experienced users of relational databases like SQL will be familiar with the terminology used to describe join operations between two SQL-table like structures (`DataFrame` objects). There are several cases to consider which are very important to understand:

- **one-to-one joins**: for example when joining two DataFrame objects on their indexes (which must contain unique values).
- **many-to-one joins**: for example when joining an index (unique) to one or more columns in a different DataFrame.
- **many-to-many joins**: joining columns on columns.

>**Note** When joining columns on columns (potentially a many-to-many join), any indexes on the passed DataFrame objects will be **discarded**.

It is worth spending some time understanding the result of the `many-to-many join` case. In SQL / standard relational algebra, if a key combination appears more than once in both tables, the resulting table will have the **Cartesian product** of the associated data. Here is a very basic example with one unique key combination:

In [151]:
left = pd.DataFrame({'keys': mklbl('K', 0, 4),
                     'A': mklbl('A', 0, 4),
                     'B': mklbl('B', 0, 4)})

In [152]:
right = pd.DataFrame({'keys': mklbl('K', 0, 4),
                     'C': mklbl('C', 0, 4),
                     'D': mklbl('D', 0, 4)}) 

In [155]:
pd.concat([left, right], axis=1)

,keys,A,B,keys,C,D
0,K0,A0,B0,K0,C0,D0
1,K1,A1,B1,K1,C1,D1
2,K2,A2,B2,K2,C2,D2
3,K3,A3,B3,K3,C3,D3


In [157]:
result = pd.merge(left, right, on='keys')

In [158]:
result

,keys,A,B,C,D
0,K0,A0,B0,C0,D0
1,K1,A1,B1,C1,D1
2,K2,A2,B2,C2,D2
3,K3,A3,B3,C3,D3


Here is a more complicated example with multiple join keys. Only the keys appearing in `left` and `right` are present (the intersection), since `how='inner'` by default.

In [159]:
left = pd.DataFrame({'key1': ['K0', 'K0', 'K1', 'K2'],
                     'key2': ['K0', 'K1', 'K0', 'K1'],
                      'A': ['A0', 'A1', 'A2', 'A3'],
                      'B': ['B0', 'B1', 'B2', 'B3']})

In [160]:
right = pd.DataFrame({'key1': ['K0', 'K1', 'K1', 'K2'],
                      'key2': ['K0', 'K0', 'K0', 'K0'],
                      'C': ['C0', 'C1', 'C2', 'C3'],
                      'D': ['D0', 'D1', 'D2', 'D3']})

In [161]:
pd.concat([left, right], axis=1)

,key1,key2,A,B,key1,key2,C,D
0,K0,K0,A0,B0,K0,K0,C0,D0
1,K0,K1,A1,B1,K1,K0,C1,D1
2,K1,K0,A2,B2,K1,K0,C2,D2
3,K2,K1,A3,B3,K2,K0,C3,D3


In [162]:
result = pd.merge(left, right, on=['key1', 'key2'])

In [163]:
result

,key1,key2,A,B,C,D
0,K0,K0,A0,B0,C0,D0
1,K1,K0,A2,B2,C1,D1
2,K1,K0,A2,B2,C2,D2


The `how` argument to merge specifies how to determine which keys are to be included in the resulting table. If a key combination **does not appear** in either the left or right tables, the values in the joined table will be NA. Here is a summary of the `how` options and their SQL equivalent names:

Merge method	SQL Join Name	Description
left	LEFT OUTER JOIN	Use keys from left frame only
right	RIGHT OUTER JOIN	Use keys from right frame only
outer	FULL OUTER JOIN	Use union of keys from both frames
inner	INNER JOIN	Use intersection of keys from both frames

In [165]:
pd.merge(left, right, how='left', on=['key1', 'key2'])

,key1,key2,A,B,C,D
0,K0,K0,A0,B0,C0,D0
1,K0,K1,A1,B1,NaN,NaN
2,K1,K0,A2,B2,C1,D1
3,K1,K0,A2,B2,C2,D2
4,K2,K1,A3,B3,NaN,NaN


In [167]:
pd.merge(left, right, how='right', on=['key1', 'key2'])

,key1,key2,A,B,C,D
0,K0,K0,A0,B0,C0,D0
1,K1,K0,A2,B2,C1,D1
2,K1,K0,A2,B2,C2,D2
3,K2,K0,NaN,NaN,C3,D3


In [168]:
pd.merge(left, right, how='outer', on=['key1', 'key2'])

,key1,key2,A,B,C,D
0,K0,K0,A0,B0,C0,D0
1,K0,K1,A1,B1,NaN,NaN
2,K1,K0,A2,B2,C1,D1
3,K1,K0,A2,B2,C2,D2
4,K2,K1,A3,B3,NaN,NaN
5,K2,K0,NaN,NaN,C3,D3


In [169]:
result = pd.merge(left, right, how='inner', on=['key1', 'key2'])
result

,key1,key2,A,B,C,D
0,K0,K0,A0,B0,C0,D0
1,K1,K0,A2,B2,C1,D1
2,K1,K0,A2,B2,C2,D2


Here is another example with duplicate join keys in DataFrames:

In [170]:
left = pd.DataFrame({'A': [1, 2], 'B': [2, 2]})
right = pd.DataFrame({'A': [4, 5, 6], 'B': [2, 2, 2]})
result = pd.merge(left, right, on='B', how='outer')

In [172]:
pd.concat([left, right], axis=1, sort=False)

,A,B,A,B
0,1.0,2.0,4,2
1,2.0,2.0,5,2
2,NaN,NaN,6,2


In [173]:
result

,A_x,B,A_y
0,1,2,4
1,1,2,5
2,1,2,6
3,2,2,4
4,2,2,5
5,2,2,6


>**Warning** Joining / merging on duplicate keys can cause a returned frame that is the multiplication of the row dimensions, which may result in memory overflow. It is the user’ s responsibility to manage duplicate values in keys before joining large DataFrames.

---
# Timeseries friendly merging

## Merging ordered data

A `merge_ordered()` function allows combining time series and other ordered data. In particular it has an optional `fill_method` keyword to fill/interpolate missing data:

In [174]:
left = pd.DataFrame({'k': ['K0', 'K1', 'K1', 'K2'],
                     'lv': [1, 2, 3, 4],
                     's': ['a', 'b', 'c', 'd']})

right = pd.DataFrame({'k': ['K1', 'K2', 'K4'],
                      'rv': [1, 2, 3]})

In [176]:
pd.merge_ordered(left, right, fill_method='ffill', left_by='s')

,k,lv,s,rv
0,K0,1.0,a,NaN
1,K1,1.0,a,1.0
2,K2,1.0,a,2.0
3,K4,1.0,a,3.0
4,K1,2.0,b,1.0
5,K2,2.0,b,2.0
6,K4,2.0,b,3.0
7,K1,3.0,c,1.0
8,K2,3.0,c,2.0
9,K4,3.0,c,3.0


## Merging asof: `merge_asof()`

A `merge_asof()` is similar to an ordered left-join except that we match on nearest key rather than equal keys. For each row in the `left` DataFrame, we select the last row in the `right` DataFrame whose `on` key is less than the left’s key. Both DataFrames must be sorted by the key.

Optionally an asof merge can perform a group-wise merge. This matches the `by` key equally, in addition to the nearest match on the `on` key.

For example; we might have `trades` and `quotes` and we want to `asof` merge them.

In [177]:
trades = pd.DataFrame({
     'time': pd.to_datetime(['20160525 13:30:00.023',
                             '20160525 13:30:00.038',
                             '20160525 13:30:00.048',
                             '20160525 13:30:00.048',
                             '20160525 13:30:00.048']),
     'ticker': ['MSFT', 'MSFT',
                'GOOG', 'GOOG', 'AAPL'],
     'price': [51.95, 51.95,
               720.77, 720.92, 98.00],
     'quantity': [75, 155,
                  100, 100, 100]},
     columns=['time', 'ticker', 'price', 'quantity'])

In [178]:
quotes = pd.DataFrame({
     'time': pd.to_datetime(['20160525 13:30:00.023',
                             '20160525 13:30:00.023',
                             '20160525 13:30:00.030',
                             '20160525 13:30:00.041',
                             '20160525 13:30:00.048',
                             '20160525 13:30:00.049',
                             '20160525 13:30:00.072',
                             '20160525 13:30:00.075']),
     'ticker': ['GOOG', 'MSFT', 'MSFT',
                'MSFT', 'GOOG', 'AAPL', 'GOOG',
                'MSFT'],
     'bid': [720.50, 51.95, 51.97, 51.99,
             720.50, 97.99, 720.50, 52.01],
     'ask': [720.93, 51.96, 51.98, 52.00,
             720.93, 98.01, 720.88, 52.03]},
     columns=['time', 'ticker', 'bid', 'ask'])
 

In [179]:
trades

,time,ticker,price,quantity
0,2016-05-25 13:30:00.023,MSFT,51.95,75
1,2016-05-25 13:30:00.038,MSFT,51.95,155
2,2016-05-25 13:30:00.048,GOOG,720.77,100
3,2016-05-25 13:30:00.048,GOOG,720.92,100
4,2016-05-25 13:30:00.048,AAPL,98.00,100


In [186]:
quotes.set_index('ticker').sort_index()

,time,bid,ask
ticker,,,
AAPL,2016-05-25 13:30:00.049,97.99,98.01
GOOG,2016-05-25 13:30:00.023,720.50,720.93
GOOG,2016-05-25 13:30:00.048,720.50,720.93
GOOG,2016-05-25 13:30:00.072,720.50,720.88
MSFT,2016-05-25 13:30:00.023,51.95,51.96
MSFT,2016-05-25 13:30:00.030,51.97,51.98
MSFT,2016-05-25 13:30:00.041,51.99,52.00
MSFT,2016-05-25 13:30:00.075,52.01,52.03


By default we are taking the asof of the quotes.

In [187]:
pd.merge_asof(trades, quotes, on='time', by='ticker')

,time,ticker,price,quantity,bid,ask
0,2016-05-25 13:30:00.023,MSFT,51.95,75,51.95,51.96
1,2016-05-25 13:30:00.038,MSFT,51.95,155,51.97,51.98
2,2016-05-25 13:30:00.048,GOOG,720.77,100,720.50,720.93
3,2016-05-25 13:30:00.048,GOOG,720.92,100,720.50,720.93
4,2016-05-25 13:30:00.048,AAPL,98.00,100,NaN,NaN


We only asof within 2ms between the quote time and the trade time.

In [188]:
pd.merge_asof(trades, quotes,
              on='time',
              by='ticker',
              tolerance=pd.Timedelta('2ms'))

,time,ticker,price,quantity,bid,ask
0,2016-05-25 13:30:00.023,MSFT,51.95,75,51.95,51.96
1,2016-05-25 13:30:00.038,MSFT,51.95,155,NaN,NaN
2,2016-05-25 13:30:00.048,GOOG,720.77,100,720.50,720.93
3,2016-05-25 13:30:00.048,GOOG,720.92,100,720.50,720.93
4,2016-05-25 13:30:00.048,AAPL,98.00,100,NaN,NaN


We only asof within 10ms between the quote time and the trade time and we exclude exact matches on time. Note that though we exclude the exact matches (of the quotes), prior quotes do propagate to that point in time.

In [189]:
pd.merge_asof(trades, quotes,
              on='time',
              by='ticker',
              tolerance=pd.Timedelta('10ms'),
              allow_exact_matches=False)

,time,ticker,price,quantity,bid,ask
0,2016-05-25 13:30:00.023,MSFT,51.95,75,NaN,NaN
1,2016-05-25 13:30:00.038,MSFT,51.95,155,51.97,51.98
2,2016-05-25 13:30:00.048,GOOG,720.77,100,NaN,NaN
3,2016-05-25 13:30:00.048,GOOG,720.92,100,NaN,NaN
4,2016-05-25 13:30:00.048,AAPL,98.00,100,NaN,NaN
